# Wall-Following with a PD Controller

If we are to follow a wall, we'll need to ability to set the differential drive robot's $\dot x$ and $\dot \theta$ velocities in its local reference frame. So we use the following `DdController` to listen for `Twist` messages with the $\dot x$ and $\dot \theta$ velocities.

```python
from geometry_msgs.msg import Twist
from rclpy.node import Node
from std_msgs.msg import Float32


class DdController(Node):
    """A ROS node to translate Twist messages to differential drive robot
    wheel speeds.
    """

    def __init__(self, twist_topic, left_topic, right_topic, r=0.5, L=0.6):
        """Constructs a DdController node.

        :param twist_topic: The topic publishing the Twist messages.
        :param left_topic: The topic to publish the left wheel speeds to.
        :param right_topic: The topic to publish the right wheel speeds to.
        :param r: The robot wheel radius.
        :param L: The robot half-axle length.
        """
        super().__init__('DdController')

        self.r = r
        self.L = L

        self.create_subscription(Twist, twist_topic, self.callback)
        self.left = self.create_publisher(Float32, left_topic)
        self.right = self.create_publisher(Float32, right_topic)

    def callback(self, msg):
        """Converts Twist messages with x and theta velocities (in local coordinates)
        to (left, right) wheel speeds.
        """
        left, right = Float32(), Float32()
        difference = msg.angular.z * 2 * self.L / self.r

        left.data = msg.linear.x
        right.data = msg.linear.x + difference

        self.left.publish(left)
        self.right.publish(right)
```

## Strategy

There are two elements to wall-following
1. Keeping the robot at the target distance from the wall.
2. Keeping the robot headed tangent to the wall. In other words, keep the direction of the minimum distance to the wall as close as possible to $\pi \over 2$ or $-\pi \over 2$ depending on whether you want to follow along the left or right side of the robot.

Our approach will attempt to address both elements. The input to our algorithm will be a [`LaserScan`](http://docs.ros.org/lunar/api/sensor_msgs/html/msg/LaserScan.html) message, and our output will be a [`Twist`](http://docs.ros.org/api/geometry_msgs/html/msg/Twist.html) message containing the lienar $\dot x$ and angular $\dot \theta$ velocities of our robot in its local reference frame.

Given the index $i$ into an array of distances, we can find the direction that index faces by

$$\phi = \left(i - \frac{l}{2}\right) \cdot \theta$$

where $l$ is the length of the array and $\theta$ is the angle between each index (The `LaserScan.angle_increment` field).

If we wish to follow a wall with the left side of our robot, we should not consider the right side of the LIDAR sweep. This can throw us off if we try going through a narrow opening, or if we're in a corridor with two walls. If we wish to accomodate those edge cases, we will need a smarter algorithm.

So given

$$d = \begin{cases}
1, &\text{ if following on left side} \\
-1, &\text{ if following on right side}
\end{cases}$$

we determine the beginning and end indices of the `LaserScan.ranges` array by

$$i_0 = l \cdot \frac{d + 1}{4}$$
$$i_n = l \cdot \frac{d + 3}{4}$$

We will use two controllers; one for each element of the wall-following. A PD controller for adjusting the distance to the wall, and a P controller for keeping the robot facing the right direction.

## PD controller for distance

First, calculate the error from the desired target distance.

$$e_d(t_n) = d_{min}(t_n) - d_{target}$$

Then the time derivative of this error is approximately

$$e_d'(t_n) = \frac{e_d(t_n) - e_d(t_{n-1})}{t_n - t_{n - 1}}$$

But if we make the assumption that $t_n - t_{n - 1}$ is constant, we can include it in our derivative gain.

So then we have the PD controller

$$U_d(t_n) = K_p e_d(t_n) + K_d \left(e_d(t_n) - e_d(t_{n - 1})\right)$$

for controlling the distance to the wall, with gains $K_p$ and $K_d$ for the proportional and derivative errors respectively.

## P controller for angle

The desired angle will depend on the side of the robot we follow the wall with. So the error in angles will be

$$e_\phi(t_n) = \phi(t_n) - d \cdot \frac{\pi}{2}$$

where, again,

$$d = \begin{cases}
1, &\text{ if following on left side} \\
-1, &\text{ if following on right side}
\end{cases}$$

Then we have the P controller

$$U_\phi(t_n) = K_\phi e_\phi(t_n)$$

## Combining the PD and P controllers

We can combine the PD and P controllers (they both control the angular velocity of the robot) to get the final PD controller

$$U(t_n) = K_p e_d(t_n) + K_d (e_d(t_n) - e_d(t_{n - 1})) + K_\phi e_\phi(t_n)$$

## Linear velocity

The above PD controller only sets the angular velocity of the robot. We still need to address the linear velocity. We do this based on two pieces of information.
1. The distance $d_f$ to an obstacle (if one exists) in front of the robot
2. The magnitude of the angle $\phi$ to the minimum distance

So we set the linear velocity to be

$$\dot x = \begin{cases}
0, &\text{ if } d_f < d_{target} \\
\frac{v}{2}, &\text{ if } d_f < 2 * d_{target} \\
\frac{v}{2}, &\text{ if } \vert\phi\vert > 1.75 \\
v, &\text{ otherwise}
\end{cases}$$

where $v$ is the desired linear velocity.

## Implementation in Python

So we have the following ROS node to implement wall-following.

```python
import numpy as np
from geometry_msgs.msg import Twist
from rclpy.node import Node
from sensor_msgs.msg import LaserScan


class WallFollowController(Node):
    """A ROS node for implementing wall following using a simple PD controller."""

    def __init__(self, lidar_topic, control_topic, speed=5.0, target_distance=5,
                 Kp=1.0, Kd=1.0, Kth=1.0, side='left'):
        """Creates a wall following node.

        :param lidar_topic: The topic publishing LIDAR messages.
        :param control_topic: The topic to publish twist messages to.
        :param Kp: The proportional gain.
        :param Kd: The derivative gain.
        :param Kth: The proportional gain for the angular velocity.
        :param side: The side of the robot to follow.
        """
        super().__init__('WallFollowController')

        self.Kp = Kp
        self.Kd = Kd
        self.Kth = Kth

        self.target_distance = target_distance
        self.speed = speed

        if side == 'left':
            self.side = 1
        elif side == 'right':
            self.side = -1
        else:
            raise ValueError('Can\'t follow side other than left or right?!')

        self.__sub = self.create_subscription(LaserScan, lidar_topic, self.callback)
        self.pub = self.create_publisher(Twist, control_topic)

        self.error = 0

    def callback(self, msg):
        """Receives LIDAR data at some frequency."""
        # BUG: If the ranges on the side we care about are all infinite,
        #      then Veranda crashes.
        l = len(msg.ranges)
        i_0 = l * (self.side + 1) // 4
        i_n = l * (self.side + 3) // 4

        min_distance = msg.ranges[i_0]
        min_i = i_0

        # Need both the min value and the index at which the min occurs.
        for i, d in enumerate(msg.ranges[i_0:i_n + 1]):
            if d < min_distance:
                min_distance = d
                min_i = i

        min_angle = (min_i - l // 2) * msg.angle_increment
        front_distance = msg.ranges[l // 2]

        diff_error = min_distance - self.target_distance - self.error
        self.error = min_distance - self.target_distance

        self.pd_controller(min_angle, front_distance, self.error, diff_error)

    def pd_controller(self, min_angle, front_distance, error, diff_error):
        """Implements a simple PD controller to keep the robot at a set distance
        and heading (tangent to obstacle)

        :param min_angle: The angle to the minimum distance.
        :param front_distance: The distance from the front scan.
        :param error: The reference error.
        :param diff_error: The time derivative of the reference error.
        """
        msg = Twist()
        # TODO: Try adding an integral term?
        # The angular velocity of the robot.
        msg.angular.z = self.side * (self.Kp * error + self.Kd * diff_error) + \
                        self.Kth * (min_angle - np.pi * self.side / 2)

        # Here, (x, y) is in the robot local coordinate system.
        # If we're about to run into something, stop.
        if front_distance < self.target_distance:
            msg.linear.x = 0.0
        # If we're less about to run into something slow down.
        elif front_distance < self.target_distance * 2:
            msg.linear.x = self.speed / 2
        # If the angle correction is too big, slow down.
        elif abs(min_angle) > 1.75:
            # TODO: This angle is still too extreme.
            msg.linear.x = self.speed / 2
        else:
            msg.linear.x = self.speed

        self.pub.publish(msg)
```

Then we use the two nodes as follows

```python
#!/usr/bin/env python3
import sys
sys.path.append('../..')

import rclpy

from robotics.nodes import GpsPlotter, NodeManager, DdController, WallFollowController


def main():
    """An example of wall following in Veranda."""
    rclpy.init()
    manager = NodeManager(policy='multi')

    controller = DdController(twist_topic='robot/control/twist',
                              left_topic='robot/left_wheel',
                              right_topic='robot/right_wheel',
                             )
    plotter = GpsPlotter('GpsPlotter', 'robot/gps', history=None)
    wall_follower = WallFollowController(lidar_topic='robot/lidar',
                                         control_topic='robot/control/twist',
                                         speed=5.0,
                                         target_distance=0.8,
                                         Kp=1.2,
                                         Kd=1.1,
                                         Kth=1.1,
                                        )

    manager.add_node(controller)
    manager.add_node(wall_follower)
    manager.add_node(plotter)

    manager.run()
    rclpy.shutdown()


if __name__ == '__main__':
    main()
```

Note that the gains $K_p$, $K_d$, and $K_\phi$ need to be tuned by hand! See the attached `wall_following_5x.mp4` for an example.